In [98]:
import os
from PIL import Image
import pytesseract
import re
from pdf2image import convert_from_path

import logging
import sys


import pandas as pd
import numpy as np

In [2]:
arvd_strs = "|".join(['Arvd~', 'Arvd-'])
clrd_strs = "|".join(['Clrd-', 'Cird-'])

def find_between(entry_text, left_text, right_text):
    left_end = re.search(left_text, entry_text)
    right_start = re.search(right_text, entry_text)
    
    if left_end and right_start:
        return entry_text[left_end.end():right_start.start()].strip()
    else:
        return None
    

In [3]:

        
def parse_entry(entry_text):
    entry_words = [w for w in entry_text.split(' ') if len(w) > 0]

    # call number is always the first word of the entry
    call_number = entry_words[0]

    # the next 'word' is always the time
    call_time = entry_words[1][:4]
    
    # call reason
    call_reason =  find_between(entry_text, call_time, 'Call Taker')
    
    # call taker is always the string between taker and location
    call_taker =  find_between(entry_text, 'Call Taker', 'Location/Addr')
    
    return [call_number, call_time, call_reason, call_taker]


def replace_none_with_value(left_list, right_list):
    return [l if not l is None else r for l, r in zip(left_list, right_list)]


In [273]:
year_str = '19-'
current_date = '01/01/2019'

parsed_addresses = pd.DataFrame(columns = ["call_number","location_address","unit"])

i_entry = 0
parsed_pages = []

for ipage in range(1,100):
    with open('../../data/Logs2019/page_{}.txt'.format(ipage),'r') as infile:
        page_text = infile.read()

    page_string_list = page_text.replace(" ","_").split("__")
    calls_idx = [i for i in range(len(page_string_list)) if re.search('19-(?=[0-9])',page_string_list[i]) != None] + [-1]

    for i in range(len(calls_idx)-1):

        incident = page_string_list[calls_idx[i]:calls_idx[i+1]]
        call = page_string_list[calls_idx[i]]
        
        # Check that call has the expected format.
        m = re.search(r'(?=[A-Z]+)',call)
        if m:
            pass
        else:
            address = None
            unit = None
            loc_series = pd.Series(incident)[pd.Series(incident).str.match(r'Location/Address*')]
            if len(loc_series) > 0:
                loc_idx = loc_series.index[0]
                if loc_idx != len(incident) -1:
                    address = incident[loc_idx +1]

            unit_series = pd.Series(incident)[pd.Series(incident).str.match(r'Unit*')]
            if len(unit_series) > 0:
                unit_idx = unit_series.index[0]
                if unit_idx != len(incident) -1:
                    unit = incident[unit_idx +1]

            parsed_addresses.loc[len(parsed_addresses)] = [call, address,unit]

parsed_addresses

,call_number,location_address,unit
0,19-1,[WIA_249]_NORTH_ST,31
1,19-4,COLD_SPRING_RD,34
2,19-5,LONGVIEW_TER,34
3,19-6,FRENTER_AVE,34
4,19-7,HARWOOD_ST,34
...,...,...,...
512,19-692,MAIN_ST,34
513,19-693,[WIA_249]_NORTH_ST,37K
514,19-694,SPRING_ST,38
515,19-695,PARK_ST,31
